<a href="https://colab.research.google.com/github/BrotherKim/Colab/blob/main/SEP531/Tokenization__Sentence_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [SEP 531] 정보검색 및 자연어처리 실습: Preprocessing (Tokenization & Sentence Segmenation)

Primary TA: 이영준

TA's E-mail: passing2961@gmail.com

본 실습은 말뭉치를 특정 unit 단위로 쪼개는 tokenization 과정에서 사용되는 모델에는 어떤 것이 있는지를 살펴보며, 한국어를 직접 다뤄보는 것을 목표로 하고 있습니다. Tokenization 은 자연어처리 태스크를 수행하기 위해 필수적으로 거쳐야하는 과정이며, 어떤 tokenization 모델을 사용하냐에 따라 성능 차이가 나타나는 것을 알 수 있습니다. 최근에는, 코퍼스를 subword 단위로 분절화하는 Byte Pair Encoding (BPE) 방법이 사용되고 있으며, 그 변형으로 BERT 에서 사용하는 Word Piece Model (WPM), Google 의 SentencePiece Model, GPT2&RoBERTa 에서 사용되는 Byte-level BPE 등이 있습니다. 한국어의 경우에는 교착어라는 특성이 있으므로, 영어와 다르게 형태소 품사를 통해 분석된 결과 문장에 대해 BPE 를 적용하는, Morepheme-aware BPE 방법을 사용하고 있습니다. 


## Contents

- What is the Tokenization?
- Variants of Tokenization Model:
  - Jamo
  - Character
  - Word
  - Morepheme
  - Subword
  - Morepheme-aware Subword
- Sentence Splitter (문장 분리기)

[link1]: https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/
[link2]: https://github.com/kakaobrain/pororo
[link3]: https://github.com/zaemyung/sentsplit
[link4]: https://arxiv.org/abs/2010.02534
[link5]: https://huggingface.co/docs/tokenizers/python/latest/index.html

## References

- What is Tokenization in NLP? Here’s All You Need To Know ([link][link1])
- `Pororo` library ([link][link2])
- 'sentsplit` library ([link][link3])
- An Empirical Study of Tokenization Strategies for Various Korean NLP Tasks ([arXiv-link][link4])
- `huggingface`'s tokenizer ([link][link5])

## What is the Tokenization?

Tokenization 은 NLP 에서 필수적인 과정으로써, 자연어 문장 혹은 문서를 기계 (인공지능 모델, 컴퓨터 등)가 이해할 수 있도록 해주는 역할을 합니다. 즉, 어떠한 자연어 문장이 주어져도 모델이 이해할 수 있도록 자연어 문장을 분절 (break down) 하는 과정을 의미합니다. 이러한 Tokenization 은 전통적인 인공지능 기법에서 부터 딥러닝 모델들에서까지 사용되고 있습니다.

#### **Definition of Tokenization**
Tokenization 은 자연어로 이루어진 문장을 인공지능 모델이 이해할 수 있도록 작은 단위로 나누는 과정을 의미하며, 여기서 작은 단위를 소위 **`token`** 이라고 부릅니다. `Token` 의 형태는 다양하며 가장 대표적으로 사용되는 단위로는 Word, Character, Subword 등이 있습니다. 한국어의 경우에는 교착어라는 특성상 Morepheme, Constant & Vowel (자모), Syllable (어절) 등이 추가로 사용되고 있습니다. 아래의 예제를 통해 어떻게 나눠지는지 살펴보도록 하겠습니다.

- 입력 문장: 나랑 쇼핑하자

|Tokenization|Tokenized Sequence|
|------|---|
|Constant and Vowel (자모 단위)|ㄴ/ㅏ/ㄹ/ㅏ/ㅇ/*/ㅅ/ㅛ/ㅍ/ㅣ/ㅇ/ㅎ/ㅏ/ㅈ/ㅏ/.|
|Syllable (음절 단위)|나/랑/*/쇼/핑/하/자/.|
|Word (어절, 단어 단위)|나랑/쇼핑하자/.|
|Morpheme (형태소 단위)| 나/랑/*/쇼핑/하/자/.|
|Subword (서브워드 단위)|_나랑/_쇼/핑하/자/.|
|Morpheme-aware Subword (형태소 지향 서브워드 단위)|_나/_랑/*/_쇼/핑/_하/_자/_.|

위에 기술된 예제처럼, 어떤 Tokenization 방법을 쓰냐에 따라서 생성되는 토큰의 형태가 매우 다양한 것을 확인할 수 있습니다. 토큰의 형태가 다양함에 따라, 모델의 학습에 사용되는 vocab 의 형태도 달라지게 되며 이는 직접적으로 성능에 영향을 주기도 합니다. 최근, Tokenization 방법들을 다양하게 바꿔가면서 한국어 관련 NLU task 에 성능 변화에 어떤 영향을 끼치는지 실험적으로 분석한 논문도 공개가 되어있습니다. 해당 논문에서는 KorQuAD 태스크에서는 Subword 단위가, KorNLI, KorSTS, NSMC, PAWS-X 태스크에서는 Morpheme-aware Subword 단위가 가장 좋은 성능을 달성한 것으로 나타나고 있습니다.

#### **What about the Korean?**

| 원형 | 피동 | 높임 | 과거 | 추측 | 전달 |   | 결과 |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| 잡 |  |  |  |  |  | +다 | 잡다 |
| 잡 | +히 |  |  |  |  | +다 | 잡히다 |
| 잡 | +히 | +시 |  |  |  | +다 | 잡히시다 |
| 잡 | +히 | +시 | +었 |  |  | +다 | 잡히셨다 |
| 잡 |  |  | +았(었) |  |  | +다 | 잡았다 |
| 잡 |  |  |  | +겠 |  | +다 | 잡겠다 |
| 잡 |  |  |  |  | +더라 |  | 잡더라 |
| 잡 | +히 |  | +었 |  |  | +다 | 잡혔다 |

- 한국어는 교착어
  - 단어에 조사가 붙어 의미와 문법적 기능이 부여
    - e.g. 그가, 그에게, 그를, 그와, 그는
  - 형태소 (morpheme): 뜻을 가진 가장 작은 말의 단위

- 한국어는 띄어쓰기가 잘 지켜지지 않음
  - "띄어쓰기를전혀하지않아도글이무슨의미인지이해할수있습니다."


[konlpy]: https://konlpy.org/en/latest/
[khaiii]: https://github.com/kakao/khaiii
[ETRI Open API]: http://aiopen.etri.re.kr/service_api.php
[Pynori]: https://github.com/gritmind/python-nori
[Mecab]: https://bitbucket.org/eunjeon/mecab-ko-dic/src/master/
[Pororo]: https://github.com/kakaobrain/pororo

- (공개) 형태소 분석기
  - [konlpy]
  - [khaiii] 
  - [ETRI Open API]
  - [Pynori]
  - [Mecab]
  - [Pororo]
  
#### **The True Reasons behind Tokenization**
Tokenization 을 과정을 통해 생성된 token 들은 vocabulary 를 구성하는 데에 사용이 됩니다. Vocabulary 는 주어진 corpus 에서 unique token 들의 set 을 의미하며, vocabulary 를 구성할 때 corpus 내에서 몇 번 등장하였는지 그 빈도수를 기준으로 상위 K 개의 token 들을 사용합니다.

Vocabulary 는 `TF-IDF` 에서는 vocabulary 내 각 단어들이 특정 feature 로 사용이 되며, 딥러닝 계열의 모델에서는 자연어로 이루어진 입력 문장을 tokenizing 하여서 index id 로 표현하는 데에 사용이 됩니다. 


#### **Which Tokenization Should We Use?**
어떤 Tokenization 을 사용하냐에 따라, 인공지능 모델의 성능이 달라지므로 효과적인 tokenization 방법을 사용하는 것이 중요합니다. 인공지능 모델의 성능이 달라지는 이유는 **Out of Vocabulary (OOV)** 단어들을 다루는 방법이 tokenization 방법마다 다르기 때문입니다. 보편적으로 쓰이는 Word, Character, 그리고 Subword 단위들의 방법이 OOV 문제를 다룰 때 어떤 문제가 발생하는지에 대해 설명하겠습니다.

- Word Tokenization:
  - OOV 문제를 완화하기 위해 rare word 에 대해 **unknown token** 으로 치환하여 사용하는 일종의 트릭을 사용하여 해결하고자 합니다. 해당 방법을 통해, 모델이 OOV 토큰의 representation meaning 을 학습이 가능합니다.
  - 그러나, 모든 rare word 들에 대해 동일한 토큰인 unknown token 으로 치환을 하므로, 
    - 실질적인 OOV token 의 의미를 잃어버릴 수 있으며
    - 동일한 representation meaning 을 지님

- Character Tokenization:
  - OOV 문제를 Word Tokenization 보다는 해결하나, 입력 문장이 길어질수록 모델의 입력으로 들어가는 문장의 길이가 길어지므로 character 간의 relationship 을 학습하기가 어려울 수 있습니다. 

- Subword Tokenization:
  - Word 단위와 Character 단위의 장점만을 반영한 방법으로, 토큰 단위가 n-gram character 입니다. 대표적으로 Byte Pair Encoding (BPE) 방법이 있습니다.

## Variants of Tokenization Model

Tokenization 모델은 어떤 단위를 token 으로 사용하냐에 따라 다릅니다. 이번 시간에는 Word, Subword 단위의 Tokenization 방법들이 어떤 결과물을 나타내는지를 살펴보도록 하겠습니다. 실습을 위해 `kakaobrain` 팀의 `Pororo` 라이브러리를 활용하겠습니다.

- `Pororo` 라이브러리 설치

In [ ]:
!pip install -q pororo

In [ ]:
from pororo import Pororo

Pororo.available_models('tokenization')

'Available models for tokenization are ([lang]: en, [model]: moses, bpe32k.en, roberta, sent_en), ([lang]: ko, [model]: bpe4k.ko, bpe8k.ko, bpe16k.ko, bpe32k.ko, bpe64k.ko, unigram4k.ko, unigram8k.ko, unigram16k.ko, unigram32k.ko, unigram64k.ko, jpe4k.ko, jpe8k.ko, jpe16k.ko, jpe32k.ko, jpe64k.ko, mecab.bpe4k.ko, mecab.bpe8k.ko, mecab.bpe16k.ko, mecab.bpe32k.ko, mecab.bpe64k.ko, char, jamo, word, mecab_ko, sent_ko), ([lang]: ja, [model]: mecab, bpe8k.ja, sent_ja), ([lang]: zh, [model]: jieba, sent_zh)'

In [ ]:
!pip install python-mecab-ko

     |████████████████████████████████| 200 kB 3.6 MB/s 
  ERROR: Failed building wheel for python-mecab-ko
  Running setup.py clean for python-mecab-ko
Failed to build python-mecab-ko
    Running setup.py install for python-mecab-ko ... done
  DEPRECATION: python-mecab-ko was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


#### **Jamo Tokenization**

In [ ]:
jamo_tok = Pororo(task='tokenization', lang='ko', model='jamo')
jamo_result = jamo_tok("스파이더맨 3편 영화가 하루 빨리 개봉했으면 좋겠다 ㅠㅠ")
print(jamo_result)

['ᄉ', 'ᅳ', 'ᄑ', 'ᅡ', 'ᄋ', 'ᅵ', 'ᄃ', 'ᅥ', 'ᄆ', 'ᅢ', 'ᆫ', '▁', '3', 'ᄑ', 'ᅧ', 'ᆫ', '▁', 'ᄋ', 'ᅧ', 'ᆼ', 'ᄒ', 'ᅪ', 'ᄀ', 'ᅡ', '▁', 'ᄒ', 'ᅡ', 'ᄅ', 'ᅮ', '▁', 'ᄈ', 'ᅡ', 'ᆯ', 'ᄅ', 'ᅵ', '▁', 'ᄀ', 'ᅢ', 'ᄇ', 'ᅩ', 'ᆼ', 'ᄒ', 'ᅢ', 'ᆻ', 'ᄋ', 'ᅳ', 'ᄆ', 'ᅧ', 'ᆫ', '▁', 'ᄌ', 'ᅩ', 'ᇂ', 'ᄀ', 'ᅦ', 'ᆻ', 'ᄃ', 'ᅡ', '▁', 'ᅲ', 'ᅲ']


#### **Character Tokenization**

In [ ]:
char_tok = Pororo(task='tokenization', lang='ko', model='char')
char_result = char_tok("스파이더맨 3편 영화가 하루 빨리 개봉했으면 좋겠다 ㅠㅠ")
print(char_result)

['스', '파', '이', '더', '맨', '▁', '3', '편', '▁', '영', '화', '가', '▁', '하', '루', '▁', '빨', '리', '▁', '개', '봉', '했', '으', '면', '▁', '좋', '겠', '다', '▁', 'ㅠ', 'ㅠ']


#### **Word Tokenization**


In [ ]:
word_tok = Pororo(task='tokenization', lang='ko', model='word')
word_tok("스파이더맨 3편 영화가 하루 빨리 개봉했으면 좋겠다 ㅠㅠ")

['스파이더맨', '3편', '영화가', '하루', '빨리', '개봉했으면', '좋겠다', 'ㅠㅠ']

#### **Morpheme Tokenization**

In [ ]:
morp_tok = Pororo(task='tokenization', lang='ko', model='mecab_ko')
morp_result = morp_tok("스파이더맨 3편 영화가 하루 빨리 개봉했으면 좋겠다 ㅠㅠ")
print(morp_result)

['스파이더맨', ' ', '3', '편', ' ', '영화', '가', ' ', '하루', ' ', '빨리', ' ', '개봉', '했', '으면', ' ', '좋', '겠', '다', ' ', 'ㅠㅠ']


#### **Subword Tokenization**

In [ ]:
subword_tok = Pororo(task='tokenization', lang='ko', model='bpe32k.ko')
subword_result = subword_tok("스파이더맨 3편 영화가 하루 빨리 개봉했으면 좋겠다 ㅠㅠ")
print(subword_result)

['▁스파이', '더', '맨', '▁3', '편', '▁영화가', '▁하루', '▁빨리', '▁개봉', '했으면', '▁좋겠다', '▁', 'ᅲ', 'ᅲ']


#### **Morpheme-aware Subword Tokenization**

In [ ]:
morp_subword_tok = Pororo(task='tokenization', lang='ko', model='mecab.bpe32k.ko')
morp_subword_result = morp_subword_tok("스파이더맨 3편 영화가 하루 빨리 개봉했으면 좋겠다 ㅠㅠ")
print(morp_subword_result)

['▁스파이더맨', '▁3', '편', '▁영화', '가', '▁하루', '▁빨리', '▁개봉', '했', '으면', '▁좋', '겠', '다', '▁', 'ㅠㅠ']


## 그 외, (참고용)

**NLTK**

NLTK (Natural Language Toolkit) 은 교육용으로 개발된 자연어 처리 및 문서분석용 파이썬 패키지입니다. 주요 기능으로는 corpus 제공, 토큰 생성, 형태소 분석, 품사 태깅 등이 있습니다.

[spacy]: https://spacy.io/
[stanford]: https://stanfordnlp.github.io/CoreNLP/
[apache]: https://opennlp.apache.org/
[allen]: https://allennlp.org/
[gensim]: https://radimrehurek.com/gensim/
[textblob]: https://textblob.readthedocs.io/en/dev/

그 외 자연어 처리 분석 패키지
- [SpaCy][spacy] 
- [Stanford Core NLP][stanford]
- [Apache OpenNLP][apache]
- [AllenNLP][allen]
- [GenSim][gensim]
- [TextBlob][textblob]


## Sentence Splitter (문장 분리기)

문서가 긴 경우에는 문장 단위로 자르는 것이 중요합니다. 이번 시간에는 `sentsplit` 라이브러리를 활용하여 실습을 진행해보겠습니다.

[sentsplit]: https://github.com/zaemyung/sentsplit
- github repo: [sentsplit]

#### **What is `sentsplit`?**
A flexible sentence segmentation library using CRF model and regex rules.

This library allows splitting of text paragraphs into sentences. It is built with the following desiderata:

- Be able to extend to new languages or "types" of sentences from data alone by learning a conditional random field (CRF) model.
- Also provide functionality to segment (or not to segment) lines based on regular expression rules (referred as segment_regexes and prevent_regexes, respectively).
- Be able to reconstruct the exact original text paragraphs from joining the segmented sentences.

All in all, the library aims to benefit from the best of both worlds: data-driven and rule-based approaches.

#### **How to use: Python Library**
```python
from sentsplit.segment import SentSplit

# use default setting
sent_splitter = SentSplit(lang_code)

# override default setting - see "Features" for detail
sent_splitter = SentSplit(lang_code, **overriding_kwargs)

# segment a single line
sentences = sent_splitter.segment(line)

# can also segment a list of lines
sentences = sent_splitter.segment([lines])
```

In [ ]:
!pip install -q sentsplit

     |████████████████████████████████| 2.6 MB 4.0 MB/s 
     |████████████████████████████████| 743 kB 60.4 MB/s 
     |████████████████████████████████| 57 kB 4.6 MB/s 


In [ ]:
from copy import deepcopy
from sentsplit.config import ko_config
from sentsplit.segment import SentSplit

w_regex_my_config = deepcopy(ko_config)
wo_regex_my_config = deepcopy(ko_config)

w_regex_my_config['segment_regexes'].append({'name': 'tilde_ending', 'regex': r'(?<=[다요])~+(?= )', 'at': 'end'})

w_sent_splitter = SentSplit('ko', **w_regex_my_config)
wo_sent_splitter = SentSplit('ko', **wo_regex_my_config)

w_result = w_sent_splitter.segment('안녕하세요~ 만나서 정말 반갑습니다~~ 잘 부탁드립니다!')
wo_result = wo_sent_splitter.segment('안녕하세요~ 만나서 정말 반갑습니다~~ 잘 부탁드립니다!')

print(f'\nWith Regex: {w_result}')
print(f'Without Regex: {wo_result}')

2021-09-08 01:23:19.457 | INFO     | sentsplit.segment:__init__:47 - SentSplit for KO loaded:
{ 'handle_multiple_spaces': True,
  'maxcut': 500,
  'mincut': 5,
  'model': 'crf_models/ko-default-05042021.model',
  'ngram': 5,
  'prevent_regexes': [ { 'name': 'liberal_url',
                         'regex': '\\b((?:[a-z][\\w\\-]+:(?:\\/{1,3}|[a-z0-9%])|www\\d{0,3}[.]|[a-z0-9.\\-]+[.][a-z]{2,4}\\/)(?:[^\\s()<>]|\\((?:[^\\s()<>]|(?:\\([^\\s()<>]+\\)))*\\))+(?:\\((?:[^\\s()<>]|(?:\\([^\\s()<>]+\\)))*\\)|[^\\s`!()\\[\\]{};:\\\'".,<>?«»“”‘’]))'},
                       { 'name': 'period_followed_by_lowercase',
                         'regex': '\\.(?= *[a-z])'}],
  'prevent_word_split': True,
  'segment_regexes': [ {'at': 'end', 'name': 'after_semicolon', 'regex': ' *;'},
                       { 'at': 'end',
                         'name': 'ellipsis',
                         'regex': '…(?![\\!\\?\\.．？！])'},
                       {'at': 'end', 'name': 'newline', 'regex': '\\n'},
          


With Regex: ['안녕하세요~', ' 만나서 정말 반갑습니다~~', ' 잘 부탁드립니다!']
Without Regex: ['안녕하세요~ 만나서 정말 반갑습니다~~', ' 잘 부탁드립니다!']


#### **그 외,**

- `sent_tokenize` 함수는 말뭉치를 문장 단위로 토큰화 역할을 수행


In [ ]:
from nltk.tokenize import sent_tokenize

text = "I am actively looking for Ph.D. students. And you are a Ph.D student."

print(sent_tokenize(text))

['I am actively looking for Ph.D. students.', 'and you are a Ph.D student.']
